In [2]:
import numpy as np
import os


num_pts = 2 	# the number of points to generate 
start=0

directory='mechanisms/File_'
#Read in the input file, change the binding energy for H, O, C
for i in range(num_pts):
    
    #Create a new folder for the new input file and write the input file
    mkdir_writer="".join((directory,str(i+start)))
    os.mkdir(mkdir_writer)
    

FileExistsError: [Errno 17] File exists: 'mechanisms/File_0'

In [6]:
import cantera as ct
import optunity

def shift_nasa(phase, index, change_in_enthalpy):
    if phase.molecular_weights[index]>min(phase.molecular_weights):
        my_species=phase.species(index)
        new_coeffs=my_species.thermo.coeffs
        T_low=my_species.thermo.min_temp
        T_high=my_species.thermo.max_temp
        
        change_in_enthalpy/=ct.gas_constant
        new_coeffs[6]=my_species.thermo.coeffs[6] + change_in_enthalpy
        new_coeffs[13]=my_species.thermo.coeffs[13] + change_in_enthalpy
        
        my_species.thermo=ct.NasaPoly2(T_low, T_high,ct.one_atm, new_coeffs)
        phase.modify_species(index,my_species)
    return   

def create_new_mechanism(phase, Delta_DeltaH):
    #cycle through the species for each phase
    #cycle through the reactions for each phase    
    for i in range(phase.n_species):
        shift_nasa(phase, i, Delta_DeltaH[i])
        
    #for i in range(phase.n_reactions):
    #    shift_Arrhenius(phase, i, Delta_Ea[i], Delta_A[i])    
    
    return

num_pts = 500   # the number of points to generate
skip = 1000    # the number of initial points of the Sobol sequence to skip
# generate Sobol sequence
x_sobol=np.zeros((num_pts, 40))
res = optunity.solvers.Sobol.i4_sobol_generate(40, num_pts, skip)

Delta_DeltaH=10 #kJ/mol
Delta_DeltaH*=1.0E6

for i,j in enumerate(zip(*res)):
    x_sobol[:,i]=j
    
surf111_ddh = Delta_DeltaH-2*x_sobol*Delta_DeltaH

for k in range(num_pts):
    #Some parameters
    T=383 #Initial temperature
    p=1 #pressure 
    # input file containing the surface reaction mechanism
    cti_file = 'multifacet_original.yaml'
    
    # import the gas model and set the initial conditions
    gas = ct.Solution(cti_file, 'gas')
    gas.TPX = T, p*ct.one_atm, 'Ar:1, CO2:0.0, CO:0'
    
    # import the (111) surface
    surf111 = ct.Interface(cti_file,'surface1', [gas])
    surf111.TP = T, p*ct.one_atm
    surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0, 'CO2(111)':0}


CanteraError: 
*******************************************************************************
CanteraError thrown by addReactions:

*******************************************************************************
InputFileError thrown by UnitSystem::convert:
Error on line 429 of ./multifacet_original.yaml:
Couldn't parse '1.7.0e-03' as a space-separated value/unit pair

|  Line |
|   424 | surface1-reactions:
|   425 | - equation: site(111) + site(111) + O2 <=> O(111) + O(111)  # Reaction 1
|   426 |   sticking-coefficient: {A: 0.23, b: 0.0, Ea: 0}
|   427 |   id: Ni(111)
|   428 | - equation: site(111) + CO2 <=> CO2(111)  # Reaction 2
>   429 >   sticking-coefficient: {A: 1.7.0e-03, b: 0.0, Ea: 0}
                                      ^
|   430 |   id: Ni(111)
|   431 | - equation: site(111) + CO <=> OC(111)  # Reaction 3
|   432 |   sticking-coefficient: {A: 0.72, b: 0.0, Ea: 0.0}
*******************************************************************************
************************